In [1]:
import numpy as np
import json
import pandas as pd
from tqdm import trange
import argparse
from modules.gem import GeM
from modules.swin.build import build_model
from modules.swin.config import get_config
from utils.train_util import set_seed
from torch.utils.data import DataLoader
from datasets.dl import GeMData, GeMClass
from datasets.config import GeMConfig, SwinConfig
import torch
import os
from tqdm import tqdm
# import yaml
import gc
import torch.nn.functional as F
from matplotlib import pyplot as plt
from IPython.core.interactiveshell import InteractiveShell
from sklearn.metrics import roc_auc_score, accuracy_score
# InteractiveShell.ast_node_interactivity = "all"

2022-05-16 21:55:59.950974: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-05-16 21:55:59.951049: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
mi = GeMConfig('cifar100')
mi.set_arch('deit')
model = GeM(mi)

load deit


Some weights of the model checkpoint at facebook/deit-base-distilled-patch16-224 were not used when initializing DeiTModel: ['distillation_classifier.weight', 'distillation_classifier.bias', 'cls_classifier.weight', 'cls_classifier.bias']
- This IS expected if you are initializing DeiTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DeiTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DeiTModel were not initialized from the model checkpoint at facebook/deit-base-distilled-patch16-224 and are newly initialized: ['deit.pooler.dense.weight', 'deit.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference

In [3]:
# pic_matrix = torch.ByteTensor(np.load("data/imageset_small.npy"))
pic_matrix = torch.ByteTensor(np.random.randint(low=0, high=255, size=(256, 3, 224, 224)))
dataset_img = torch.LongTensor(np.random.randint(low=0, high=256, size=(1280, 1)))
dataset_label = torch.LongTensor(np.random.randint(low=0, high=100, size=(1280, 1)))

In [4]:
train_dataset = GeMClass(pic_matrix, torch.cat([dataset_img, dataset_label], dim=-1))
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# input_label = torch.zeros((32), dtype=torch.long).to(0)
input_label = torch.randint(low=0, high=100, size=(32, 1)).reshape(-1)

In [5]:
optimizer = torch.optim.AdamW(params=model.parameters(), lr=5e-4)

In [ ]:
for epoch in range(2):
    model.train()
    model.zero_grad()
    index = 0
    steps_one_epoch = len(train_data_loader)
    enum_dataloader = tqdm(train_data_loader, total=steps_one_epoch, desc="EP-{} train".format(epoch))
    loss_list = []
    for data in enum_dataloader:
    #     if index >= steps_one_epoch:
    #         break

        model_in = data[:, :-1] / 255.0
#         pred = model.st(model_in.reshape(-1, 3, 224, 224))
#         print(pred.size())
        pred = model.predict_class(model_in, 224)
        loss = F.cross_entropy(pred, data[:, -1])
        loss_list.append(loss)

        loss.backward()
        optimizer.step()
        model.zero_grad()

        enum_dataloader.set_description("EP-{} train loss: {}".format(epoch, loss))
        enum_dataloader.refresh()
        index += 1
    
    print('epoch {} end'.format(epoch))


EP-0 train loss: 4.613223075866699:   5%|█████▎                                                                                                     | 2/40 [01:09<21:42, 34.29s/it]